In [ ]:
pip install --user pandas-profiling

In [ ]:
pip install --user yellowbrick

In [ ]:
pip install --user kmodes

In [ ]:
import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns 

from sklearn.metrics import silhouette_score, silhouette_samples
import sklearn.metrics
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.mixture import GaussianMixture

import pandas_profiling
import itertools

import scipy

from yellowbrick.cluster import SilhouetteVisualizer, InterclusterDistance, KElbowVisualizer


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# This will ensure that matplotlib figures don't get cut off when saving with savefig()
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
from google.colab import files
uploaded = files.upload()
import io
data = io.BytesIO(uploaded['data_clean.csv'])
import pandas as pd
df = pd.read_csv('data_clean.csv')

#read the data and describe info
df.head()
df.tail()

Saving data_clean.csv to data_clean.csv


,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,basin,region,district_code,lga,ward,population,public_meeting,scheme_management,permit,construction_year,extraction_type_class,management,payment_type,water_quality,quality_group,quantity,source,waterpoint_type,10_yrs,status_group
0,69572,6000.0,2011-03-14,Romam Catholic Church,1390,Roman,34.938093,-9.856322,Lake Nyasa,Iringa,5,Ludewa,Mundindi,109,TRUE,VWC,FALSE,1999,gravity,vwc,annually,soft,good,enough,spring,communal standpipe,90s,0
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Lake Victoria,Mara,2,Serengeti,Natta,280,unknown,Other,TRUE,2010,gravity,wug,never pay,soft,good,insufficient,rainwater harvesting,communal standpipe,10s,0
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Pangani,Manyara,4,Simanjiro,Ngorika,250,TRUE,VWC,TRUE,2009,gravity,vwc,per bucket,soft,good,enough,dam,communal standpipe multiple,00s,0
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Ruvuma / Southern Coast,Mtwara,63,Nanyumbu,Nanyumbu,58,TRUE,VWC,TRUE,1986,submersible,vwc,never pay,soft,good,dry,machine dbh,communal standpipe multiple,80s,2
4,19728,0.0,2011-07-13,Action Aid,0,Artisan,31.130847,-1.825359,Lake Victoria,Kagera,1,Karagwe,Nyakasimbi,281,TRUE,Unknown,TRUE,1997,gravity,other,never pay,soft,good,seasonal,rainwater harvesting,communal standpipe,1997,0


,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,basin,region,district_code,lga,ward,population,public_meeting,scheme_management,permit,construction_year,extraction_type_class,management,payment_type,water_quality,quality_group,quantity,source,waterpoint_type,10_yrs,status_group
59395,60739,10.0,2013-05-03,Germany Republi,1210,CES,37.169807,-3.253847,Pangani,Kilimanjaro,5,Hai,Masama Magharibi,125,TRUE,Water Board,TRUE,1999,gravity,water board,per bucket,soft,good,enough,spring,communal standpipe,90s,0
59396,27263,4700.0,2011-05-07,Cefa Church,1212,Cefa,35.249991,-9.070629,Rufiji,Iringa,4,Njombe,Ikondo,56,TRUE,VWC,TRUE,1996,gravity,vwc,annually,soft,good,enough,river,communal standpipe,90s,0
59397,37057,0.0,2011-04-11,NaN,0,NaN,34.017087,-8.750434,Rufiji,Mbeya,7,Mbarali,Chimala,281,TRUE,VWC,FALSE,1997,handpump,vwc,monthly,fluoride,fluoride,enough,machine dbh,hand pump,1997,0
59398,31282,0.0,2011-03-08,Malec,0,Musa,35.861315,-6.378573,Rufiji,Dodoma,4,Chamwino,Mvumi Makulu,281,TRUE,VWC,TRUE,1997,handpump,vwc,never pay,soft,good,insufficient,shallow well,hand pump,1997,0
59399,26348,0.0,2011-03-23,World Bank,191,World,38.104048,-6.747464,Wami / Ruvu,Morogoro,2,Morogoro Rural,Ngerengere,150,TRUE,VWC,TRUE,2002,handpump,vwc,on failure,salty,salty,enough,shallow well,hand pump,00s,0


In [ ]:
#Profile report of the clean dataset
display(df)
df.profile_report()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,basin,region,...,extraction_type_class,management,payment_type,water_quality,quality_group,quantity,source,waterpoint_type,10_yrs,status_group
0,69572,6000.0,2011-03-14,Romam Catholic Church,1390,Roman,34.938093,-9.856322,Lake Nyasa,Iringa,...,gravity,vwc,annually,soft,good,enough,spring,communal standpipe,90s,0
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Lake Victoria,Mara,...,gravity,wug,never pay,soft,good,insufficient,rainwater harvesting,communal standpipe,10s,0
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Pangani,Manyara,...,gravity,vwc,per bucket,soft,good,enough,dam,communal standpipe multiple,00s,0
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Ruvuma / Southern Coast,Mtwara,...,submersible,vwc,never pay,soft,good,dry,machine dbh,communal standpipe multiple,80s,2
4,19728,0.0,2011-07-13,Action Aid,0,Artisan,31.130847,-1.825359,Lake Victoria,Kagera,...,gravity,other,never pay,soft,good,seasonal,rainwater harvesting,communal standpipe,1997,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,60739,10.0,2013-05-03,Germany Republi,1210,CES,37.169807,-3.253847,Pangani,Kilimanjaro,...,gravity,water board,per bucket,soft,good,enough,spring,communal standpipe,90s,0
59396,27263,4700.0,2011-05-07,Cefa Church,1212,Cefa,35.249991,-9.070629,Rufiji,Iringa,...,gravity,vwc,annually,soft,good,enough,river,communal standpipe,90s,0
59397,37057,0.0,2011-04-11,NaN,0,NaN,34.017087,-8.750434,Rufiji,Mbeya,...,handpump,vwc,monthly,fluoride,fluoride,enough,machine dbh,hand pump,1997,0
59398,31282,0.0,2011-03-08,Malec,0,Musa,35.861315,-6.378573,Rufiji,Dodoma,...,handpump,vwc,never pay,soft,good,insufficient,shallow well,hand pump,1997,0


Summarize dataset:   0%|          | 0/41 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
df.dtypes

In [ ]:
df['funder'] = df.funder.astype('category')
df['funder'] = df['funder'].cat.codes

In [ ]:
df['installer'] = df.installer.astype('category')
df['installer'] = df['installer'].cat.codes

In [ ]:
df['basin'] = df.basin.astype('category')
df['basin'] = df['basin'].cat.codes

In [ ]:
df['region'] = df.region.astype('category')
df['region'] = df['region'].cat.codes

In [ ]:
df['lga'] = df.lga.astype('category')
df['lga'] = df['lga'].cat.codes

In [ ]:
df['ward'] = df.ward.astype('category')
df['ward'] = df['ward'].cat.codes

In [ ]:
df['public_meeting'] = df.public_meeting.astype('category')
df['public_meeting'] = df['public_meeting'].cat.codes

In [ ]:
df['scheme_management'] = df.scheme_management.astype('category')
df['scheme_management'] = df['scheme_management'].cat.codes

In [ ]:
df['permit'] = df.permit.astype('category')
df['permit'] = df['permit'].cat.codes

In [ ]:
df['extraction_type_class'] = df.extraction_type_class.astype('category')
df['extraction_type_class'] = df['extraction_type_class'].cat.codes

In [ ]:
df['management'] = df.management.astype('category')
df['management'] = df['management'].cat.codes

In [ ]:
df['payment_type'] = df.payment_type.astype('category')
df['payment_type'] = df['payment_type'].cat.codes

In [ ]:
df['water_quality'] = df.water_quality.astype('category')
df['water_quality'] = df['water_quality'].cat.codes

In [ ]:
df['quality_group'] = df.quality_group.astype('category')
df['quality_group'] = df['quality_group'].cat.codes

In [ ]:
df['quantity'] = df.quantity.astype('category')
df['quantity'] = df['quantity'].cat.codes

In [ ]:
df['source'] = df.source.astype('category')
df['source'] = df['source'].cat.codes

In [ ]:
df['waterpoint_type'] = df.waterpoint_type.astype('category')
df['waterpoint_type'] = df['waterpoint_type'].cat.codes

In [ ]:
import datetime
df['date_recorded'] = pd.to_datetime(df['date_recorded'])

df['date_recorded'] = df.date_recorded.astype('category')
df['date_recorded'] = df['date_recorded'].cat.codes

In [ ]:
df['10_yrs'] = df['10_yrs'].astype('category')
df['10_yrs'] = df['10_yrs'].cat.codes

In [ ]:
#df = df.drop(['latitude'], axis=1)
#df = df.drop(['longitude'], axis=1)
#df = df.drop(['10_yrs'], axis=1)

In [ ]:
df.dtypes

id                         int64
amount_tsh               float64
date_recorded              int16
funder                     int16
gps_height                 int64
installer                  int16
longitude                float64
latitude                 float64
basin                       int8
region                      int8
district_code              int64
lga                         int8
ward                       int16
population                 int64
public_meeting              int8
scheme_management           int8
permit                      int8
construction_year          int64
extraction_type_class       int8
management                  int8
payment_type                int8
water_quality               int8
quality_group               int8
quantity                    int8
source                      int8
waterpoint_type             int8
10_yrs                      int8
status_group               int64
dtype: object

In [ ]:
k_means = KMeans(init='k-means++', n_clusters=5, n_init=10, random_state=42)
k_means.fit(df)

In [ ]:
k_means.labels_

In [ ]:
# Let's look at the centers
k_means.cluster_centers_

In [ ]:
#plot the clusters

plt.style.use('default');

plt.figure(figsize=(16, 10));
plt.grid(True);

sc = plt.scatter(df.iloc[:, 0], df.iloc[:, 1], s=200, c=k_means.labels_);
#plt.scatter(k_means.cluster_centers_[:, 0], k_means.cluster_centers_[:, 1], marker='x', s=500, c="black")
plt.title("K-Means (K=5)", fontsize=20);
plt.xticks(fontsize=18);
plt.yticks(fontsize=18);

for label in k_means.labels_:
    plt.text(x=k_means.cluster_centers_[label, 0], y=k_means.cluster_centers_[label, 1], s=label, fontsize=32, 
             horizontalalignment='center', verticalalignment='center', color='black',
             bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.1', alpha=0.02));

plt.savefig('water_pump_kmeans-5.png');

In [ ]:
#internal validation

# WCSS == Inertia
k_means.inertia_

In [ ]:
silhouette_score(df, k_means.labels_)

In [ ]:
plt.style.use('default');

sample_silhouette_values = silhouette_samples(df, k_means.labels_)
sizes = 200*sample_silhouette_values

plt.figure(figsize=(16, 10));
plt.grid(True);

plt.scatter(df.iloc[:, 0], df.iloc[:, 1], s=sizes, c=k_means.labels_)
plt.scatter(k_means.cluster_centers_[:, 0], k_means.cluster_centers_[:, 1], marker='x', s=500, c="black")

plt.title("K-Means (Dot Size = Silhouette Distance)", fontsize=20);
plt.xlabel('Status_group', fontsize=22);
plt.ylabel('population', fontsize=22);
plt.xticks(fontsize=18);
plt.yticks(fontsize=18);

plt.savefig('water_pump_kmeans-5-silhouette-size.png');

In [ ]:
X = df.copy()
X = df[['region', 'district_code']]

In [ ]:
db = DBSCAN(eps=0.3, min_samples=3)
db.fit(X)

DBSCAN(algorithm='auto', eps=0.3, leaf_size=30, metric='euclidean',
       metric_params=None, min_samples=3, n_jobs=None, p=None)

In [ ]:
print(silhouette_score(X, db.labels_))



0.999983164983165


In [ ]:
db.labels_

array([ 0,  1,  2, ..., 44, 22, 48])

In [ ]:
plt.figure();

unique_labels = set(db.labels_)
colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))];

for k in unique_labels:
    if k == -1:        # Black used for noise.
        col = [0, 0, 0, 1]
    else:
        col = colors[k]

    xy = X[db.labels_ == k]
    plt.plot(xy.iloc[:, 0], xy.iloc[:, 1], 'o', markerfacecolor=tuple(col), markeredgecolor='k', markersize=10);

    
plt.title('');
plt.title("DBSCAN (n_clusters = {:d}, black = outliers)".format(len(unique_labels)));
plt.xlabel('region');
plt.ylabel('district_code');

In [ ]:
# Instantiate the clustering model and visualizer
visualizer = InterclusterDistance(k_means)

visualizer.fit(df) # Fit the training data to the visualizer
visualizer.poof() # Draw/show/poof the data
plt.savefig('water_pump-kmeans-5-tsne.png', transparent=False);

In [ ]:
#elbow method

inertias = {}
silhouettes = {}
for k in range(2, 11):
    kmeans = KMeans(init='k-means++', n_init=10, n_clusters=k, max_iter=1000, random_state=42).fit(df)
    inertias[k] = kmeans.inertia_ # Inertia: Sum of distances of samples to their closest cluster center
    silhouettes[k] = silhouette_score(df, kmeans.labels_, metric='euclidean')
    

plt.figure();
plt.grid(True);
plt.plot(list(inertias.keys()), list(inertias.values()));
plt.title('K-Means, Elbow Method')
plt.xlabel("Number of clusters, K");
plt.ylabel("Inertia");
plt.savefig('Water_pump-kmeans-elbow-interia.png');


plt.figure();
plt.grid(True);
plt.plot(list(silhouettes.keys()), list(silhouettes.values()));
plt.title('K-Means, Elbow Method')
plt.xlabel("Number of clusters, K");
plt.ylabel("Silhouette");
plt.savefig('water_pump-kmeans-elbow-silhouette.png');

#elbow method using yellow brick package

model = KMeans(init='k-means++', n_init=10, max_iter=1000, random_state=42)
KElbowVisualizer(model, k=(2,11), metric='silhouette', timings=False).fit(df).poof();
KElbowVisualizer(model, k=(2,11), metric='calinski_harabaz', timings=False).fit(df).poof();
KElbowVisualizer(model, k=(2,11), metric='distortion', timings=False).fit(df).poof();

In [ ]:
#interpreting the clusters

#means
k_means.cluster_centers_

for label in set(k_means.labels_):
    print('\nCluster {}:'.format(label))
    df_tmp = df[k_means.labels_==label].copy()
    df_tmp.loc['mean'] = df_tmp.mean()
    df_tmp.tail(13)


In [ ]:
#exemplars
from scipy.spatial import distance

for i, label in enumerate(set(k_means.labels_)):    
    df_tmp = df[k_means.labels_==label].copy()
    
    exemplar_idx = distance.cdist([k_means.cluster_centers_[i]], df_tmp).argmin()
    exemplar = pd.DataFrame(df_tmp.iloc[exemplar_idx])
   
    print('\nCluster {}:'.format(label))
    exemplar

In [ ]:
#snake plot

dat = df.copy()

dat['Cluster'] = k_means.labels_

datamart_melt = pd.melt(dat.reset_index(),
id_vars=['Cluster'],
var_name='Feature',
value_name='Value')

plt.title('Snake Plot, K-Means, K=5')
sns.lineplot(x="Feature", y="Value", hue='Cluster', data=datamart_melt)
plt.savefig('water_pump-kmeans-5-snake.png', transparent=False);

In [ ]:
#relative importance plot
cluster_avg = dat.groupby(['Cluster']).mean()
population_avg = dat.drop(['Cluster'], axis=1).mean()

relative_imp = cluster_avg - population_avg

plt.figure(figsize=(8, 2));
plt.title('Relative importance of features');
sns.heatmap(data=relative_imp, annot=True, fmt='.2f', cmap='RdYlGn');
plt.savefig('water_pump-kmeans-5-importance.png', transparent=False);

In [ ]:
#experiment with lots of Ks
def do_kmeans(df, k):
    k_means = KMeans(init='k-means++', n_clusters=k, n_init=10, random_state=42)
    k_means.fit(df)
    wcss = k_means.inertia_
    sil = silhouette_score(df, k_means.labels_)
    
    plt.style.use('default');

    sample_silhouette_values = silhouette_samples(df, k_means.labels_)
    sizes = 200*sample_silhouette_values

    plt.figure(figsize=(16, 10));
    plt.grid(True);

    plt.scatter(df.iloc[:, 0], df.iloc[:, 1], s=sizes, c=k_means.labels_)
    plt.scatter(k_means.cluster_centers_[:, 0], k_means.cluster_centers_[:, 1], marker='x', s=500, c="black")

    plt.title("K-Means (K={}, WCSS={:.2f}, Sil={:.2f})".format(k, wcss, sil), fontsize=20);
    plt.xticks(fontsize=18);
    plt.yticks(fontsize=18);
    plt.savefig('water_pump-kmeans-auto-{}-silhouette-size.png'.format(k));
    plt.show()
    
    
    visualizer = SilhouetteVisualizer(k_means)
    visualizer.fit(df)
    visualizer.poof()
    fig = visualizer.ax.get_figure()
    fig.savefig('water_pump-kmeans-auto-{}-silhouette-plot.png'.format(k), transparent=False);
    
    print("K={}, WCSS={:.2f}, Sil={:.2f}".format(k, wcss, sil))

for k in range(2, 21):
    do_kmeans(df, k)

In [ ]:
#based on silhoutte scores above 0.5
k_means = KMeans(init='k-means++', n_clusters=5, n_init=10, random_state=42)
k_means.fit(df)
sample_silhouette_values = silhouette_samples(df, k_means.labels_)

df_tmp = df.copy()
df_tmp['Cluster ID'] = k_means.labels_

In [ ]:
df_tmp['Silhouette'] = sample_silhouette_values
df_tmp = df_tmp.sort_values(['Silhouette'])
df_tmp.head()
df_tmp.tail()

In [ ]:
plt.figure(figsize=(16, 10));
plt.grid(True);

plt.scatter(df_tmp['population'], df_tmp['amount_tsh'], sizes=200*(df_tmp['Silhouette']+0.05), c=df_tmp['Cluster ID'])

plt.title("K-Means (K = {}, Sil={:.2f})".format(k_means.n_clusters, silhouette_score(df, kmeans.labels_, metric='euclidean')), fontsize=20);
plt.xlabel('population (K)', fontsize=22);
plt.ylabel('amount_tsh', fontsize=22);
plt.xticks(fontsize=18);
plt.yticks(fontsize=18);

for label in k_means.labels_:
    plt.text(x=k_means.cluster_centers_[label, 0], y=k_means.cluster_centers_[label, 1], s=label, fontsize=32, 
             horizontalalignment='center', verticalalignment='center', color='black',
             bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.1', alpha=0.02));
    
for i, txt in enumerate(df_tmp.index.values):
    sil = df_tmp['Silhouette'].iloc[i]
    if sil < 0.05:
        plt.text(x=df_tmp['population'].iloc[i], y=df_tmp['amount_tsh'].iloc[i], s=txt, fontsize=22)

plt.savefig('water_pump-kmeans-{}-silhouette-size-labels.png'.format(k_means.n_clusters));

In [ ]:
visualizer = SilhouetteVisualizer(k_means)
visualizer.fit(df)
visualizer.poof()
fig = visualizer.ax.get_figure()
fig.savefig('water_pump-kmeans-{}-silhouette-f.png'.format(k_means.n_clusters), transparent=False);

In [ ]:
df_tmp.to_csv('k-means_clustering_data.csv')

In [ ]:
# Compute DBSCAN
db = DBSCAN(eps=0.5, min_samples=5, metric='euclidean', metric_params=None, algorithm='auto', leaf_size=30, p=None, n_jobs=None)
db.fit(df)

DBSCAN()

In [ ]:
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)
print("Silhouette Coefficient: %0.3f"
      % silhouette_score(df, labels))


Estimated number of clusters: 0
Estimated number of noise points: 59400


ValueError: Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)

In [ ]:
silhouette_score(df, labels)

In [ ]:
plt.figure(figsize=(16, 10));
plt.grid(True);

unique_labels = set(db.labels_)
colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))];

for k in unique_labels:
    if k == -1:        # Black used for noise.
        col = [0, 0, 0, 1]
    else:
        col = colors[k]

    xy = X[db.labels_ == k]
    plt.plot(xy.iloc[:, 0], xy.iloc[:, 1], 'o', markerfacecolor=tuple(col), markeredgecolor='k', markersize=14);

    
plt.title('');
plt.title("DBSCAN (n_clusters = {:d}, black = outliers)".format(len(unique_labels)), fontsize=20);
plt.xticks(fontsize=18);
plt.yticks(fontsize=18);
plt.savefig('water_pump-dbscan-03.png', transparent=False);

In [ ]:
#elbow method
silhouettes = {}
for eps in np.arange(0.1, 0.6, 0.1):
    db = DBSCAN(eps=eps, min_samples=3).fit(df)
    silhouettes[eps] = silhouette_score(df, db.labels_, metric='euclidean')
    

plt.figure();
plt.plot(list(silhouettes.keys()), list(silhouettes.values()));
plt.title('DBSCAN, Elbow Method')
plt.xlabel("Eps");
plt.ylabel("Silhouette");
plt.grid(True);
plt.savefig('water_pump-dbscan-03-silhouette.png', transparent=False);

In [ ]:
#parameter exploration
def do_dbscan(df, eps, min_samples):
    db = DBSCAN(eps=eps, min_samples=min_samples).fit(df)

    unique_labels = set(db.labels_)
    n_clusters = len(unique_labels)
    
    if n_clusters <= 1:
        print('eps={}, min_samples={}, n_clusters <= 1. Returning.'.format(eps, min_samples))
        return
    
    sil = silhouette_score(df, db.labels_)
    print("eps={}, min_samples={}, n_clusters={}, sil={}".format(eps, min_samples, n_clusters, sil))
    
    plt.figure(figsize=(16, 10));
    plt.grid(True);   
    
    colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))];

    for k in unique_labels:
        if k == -1:        # Black used for noise.
            col = [0, 0, 0, 1]
        else:
            col = colors[k]

        xy = X[db.labels_ == k]
        plt.plot(xy.iloc[:, 0], xy.iloc[:, 1], 'o', markerfacecolor=tuple(col), markeredgecolor='k', markersize=14);


    plt.title('');
    plt.title("DBSCAN (eps={}, min_samples={}, n_clusters = {:d}, sil={:.3f})".format(eps, min_samples, n_clusters, sil), fontsize=20);
    plt.xticks(fontsize=18);
    plt.yticks(fontsize=18);
    plt.savefig('water_pump-dbscan-auto-{}-{}.png'.format(eps, min_samples), transparent=False);
    

epss = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
min_samples = range(1, 10)

for prod in list(itertools.product(epss, min_samples)):
    do_dbscan(df, prod[0], prod[1])

In [ ]:
#hierarchial clustering
import scipy.cluster.hierarchy as sch
plt.figure(figsize=(10,7))
plt.title=("Dendrogram")
dend=sch.dendrogram(sch.linkage(df, method='ward'))

<Figure size 720x504 with 0 Axes>

In [ ]:
from sklearn.cluster import AgglomerativeClustering
cluster = AgglomerativeClustering(n_clusters=2, affinity= 'euclidean', linkage= 'ward')
cluster.fit_predict(df)